In [ ]:
import os 
import time 
import json,math,sys,re
import pandas as pd 

In [ ]:
qp_folder = "/Users/amycweng/Downloads/QP"
from dotenv import load_dotenv
env_path = '/Users/amycweng/DH/deepseek.env'
load_dotenv(dotenv_path=env_path)
DEEPSEEK_KEY = os.getenv('DEEPSEEK_KEY')
from openai import OpenAI
client = OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com")

In [ ]:
inputfname = "preE_qp"
fname = f"{inputfname}_DS"
data = pd.read_csv(f"{qp_folder}/{inputfname}.csv")
data = data.drop(columns=['0','1','2','3','4','5','Unnamed: 7'])
data = data.rename(columns = {'6':'text','8':'verse_text','9':'my_label'})
data = data.to_dict(orient="records")

In [ ]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
fname = f"{inputfname}_DS"
data = pd.read_csv(f"{qp_folder}/{inputfname}.csv")
data = data.to_dict(orient="records")

In [90]:
for idx, item in enumerate(data): 
    item['index'] = idx 
with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'r') as f: 
    labels = json.load(f) 
data[196], len(data), len(labels),labels['196']

({'text': 'Death is swalowed into victory.',
  'verse_text': 'And when this mortal hath put on immortality, then shall come to pass the saying that is written: Death is swallowed up in victory.',
  'my_label': 'TRUE',
  'index': 196},
 4842,
 3000,
 'TRUE')

In [84]:
from tqdm import tqdm 
start = 2512 #
end = 3000
my_qp = data[start:end] 

for qp_pair in tqdm(my_qp):
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are an English and Latin scholar working on Biblical language in early modern texts. You determine whether the first sentence is a quotation/paraphrase of a given Biblical verse in the SAME LANGUAGE. You give a single TRUE or FALSE output FOR EACH USER MESSAGE without any reasoning."},
            {"role": "user",  "content": f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"}
        ],
        stream=False
    )
    labels[qp_pair['index']] = response.choices[0].message.content

with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f) # 13mins for 100 entries; 50mins for 400 entries  # 13 secs per entry

100%|██████████| 488/488 [1:25:42<00:00, 10.54s/it]


In [91]:
for idx, entry in labels.items(): 
    entry = entry.strip()
    if entry not in ["TRUE","FALSE"]: 
        qp_pair = data[int(idx)]
        print(idx)
        print(qp_pair)
        print(entry)
        # response = client.chat.completions.create(
        #     model="deepseek-chat",
        #     messages=[
        #         {"role": "system", "content": "You are an English and Latin scholar working on Biblical language in early modern texts. You determine whether the first sentence is a quotation/paraphrase of a given Biblical verse in the SAME LANGUAGE. You give a single TRUE or FALSE output FOR EACH USER MESSAGE without any reasoning."},
        #         {"role": "user",  "content": f"SENTENCE: '{qp_pair['text']}'; 'VERSE: {qp_pair['verse_text']}'"}
        #     ],
        #     stream=False
        # )
        # labels[str(qp_pair['index'])] = response.choices[0].message.content
        # print(response.choices[0].message.content)

In [86]:
with open(f"/Users/amycweng/DH/Early-Modern-Sermons/assets/QP_labels/{fname}.json",'w') as f: 
    json.dump(labels,f)

len(labels)

3001